In [11]:
# get ratios for segment analysis 
# use BL of whole campaign 
# date created: 3/2/2020
# author: sofia chelpon 


In [3]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns

import datetime as dt 

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [4]:
#### ------------------- LOAD IN 
# data overall 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df_replace_twp_noRF06.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df_twp_noRF06.pkl")

# segment data
awas_segments = pd.read_pickle("./awas_segments_noRF06.pkl")
awas_segments = awas_segments.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns ='GGLON').drop(columns = 'Notes')
# segment data 
toga_segments = pd.read_pickle("./toga_segments_noRF06.pkl")
toga_segments = toga_segments.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns ='GGLON')

In [5]:
# save positions of each segment 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df_replace_twp_noRF06.pkl")
awas_df_ut = awas_df.loc[(awas_df['GGALT'] < 14000) & (awas_df['GGALT'] > 12000)]

awas_positions = awas_df_ut.transpose()
awas_positions = awas_positions[:7]
awas_positions.to_pickle("./awas_positions_noRF06.pkl")

In [6]:
##### --------------------- get campaign average BL means 
# AWAS 
awas_bl = pd.Series()
awas_bl = awas_df.loc[(awas_df['GGALT'] < 2000)].mean()
awas_bl = awas_bl.drop('GGALT').drop('GGLAT').drop('GGLON')
awas_bl.index.name = "Trace_Gas"

# TOGA
toga_bl = pd.Series()
toga_bl = toga_df.loc[(toga_df['GGALT'] < 2000)].mean()
toga_bl = toga_bl.drop('GGALT').drop('GGLAT').drop('GGLON')
toga_bl.index.name = "Trace_Gas"

/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


In [7]:
######## ------------- GET AWAS RATIOS 
# thing to fill with ratios 
awas_seg_ratios = pd.DataFrame()
for col in awas_df.columns: 
    awas_seg_ratios[col] = ''
awas_seg_ratios = awas_seg_ratios.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns = 'GGLON').drop(columns = 'Notes').drop(columns = 'Intrument')
    
# loop through every segment and get ratio for it 
for idx in (np.arange(0, len(awas_segments))):
    # find segment, get info 
    awas_ut = awas_segments.iloc[idx]
    idx_info = pd.Series({'Time_UTC': awas_ut['Time_UTC'], 'Flight': awas_ut['Flight']})
    # get ratio 
    awas_ut = awas_ut.drop('Time_UTC').drop('Intrument').drop('Flight')
    idx_rat = awas_ut/awas_bl
    # merge to info 
    to_append = idx_info.append(idx_rat)
    to_append = pd.DataFrame(to_append).transpose()
    # append
    awas_seg_ratios = awas_seg_ratios.append(to_append, ignore_index = True)

In [8]:
######## ------------- GET TOGA RATIOS 
# thing to fill with ratios 
toga_seg_ratios = pd.DataFrame()
for col in toga_df.columns: 
    toga_seg_ratios[col] = ''
toga_seg_ratios = toga_seg_ratios.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns = 'GGLON').drop(columns = 'Notes').drop(columns = 'Intrument')
    
# loop through every segment and get ratio for it 
for idx in (np.arange(0, len(toga_segments))):
    # find segment, get info 
    toga_ut = toga_segments.iloc[idx]
    idx_info = pd.Series({'Time_UTC': toga_ut['Time_UTC'], 'Flight': toga_ut['Flight']})
    # get ratio 
    toga_ut = toga_ut.drop('Time_UTC').drop('Intrument').drop('Flight')
    idx_rat = toga_ut/toga_bl
    # merge to info 
    to_append = idx_info.append(idx_rat)
    to_append = pd.DataFrame(to_append).transpose()
    # append
    toga_seg_ratios = toga_seg_ratios.append(to_append, ignore_index = True)


In [9]:
### add attribute to each species (BL, tropo, and UT lifetimes)
fill = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan]], columns=['Trace_Gas', 'BL_tau', 'TROPO_tau', 'UT_tau'])
fill = fill.append(fill)

# --------- AWAS 
# read in from xls file to make dataframe - awas
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_08042020.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
awas_lifetimes = fill.append(awas_lifetimes)
awas_lifetimes = awas_lifetimes.reset_index().drop('index', axis = 1)

# --------- TOGA
# read in from xls file to make dataframe - toga 
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_08042020.xlsx'
toga_lifetimes = pd.read_excel(mypath2)
toga_lifetimes = fill.append(toga_lifetimes)
toga_lifetimes = toga_lifetimes.reset_index().drop('index', axis = 1)

In [10]:
# combine data from both instruments to one dataframe 
# --------- AWAS 
awas_seg_ratios = awas_seg_ratios.transpose()
awas_seg_ratios.insert(0, 'Instrument', 'AWAS')

# add lifetimes 
awas_seg_ratios.insert(1, 'BL_tau', awas_lifetimes['BL_tau'].values)
awas_seg_ratios.insert(2, 'TROPO_tau', awas_lifetimes['TROPO_tau'].values)
awas_seg_ratios.insert(3, 'UT_tau', awas_lifetimes['UT_tau'].values)

# --------- TOGA 
toga_seg_ratios = toga_seg_ratios.transpose()
toga_seg_ratios.insert(0, 'Instrument', 'TOGA')

# add lifetimes 
toga_seg_ratios.insert(1, 'BL_tau', toga_lifetimes['BL_tau'].values)
toga_seg_ratios.insert(2, 'TROPO_tau', toga_lifetimes['TROPO_tau'].values)
toga_seg_ratios.insert(3, 'UT_tau', toga_lifetimes['UT_tau'].values)

In [11]:
# combine data from both instruments to one dataframe 
toga_seg_ratios = toga_seg_ratios.drop('Time_UTC', axis = 0)
toga_seg_ratios = toga_seg_ratios.drop('Flight', axis = 0)

ratios_comb = awas_seg_ratios.append(toga_seg_ratios)
ratios_comb

,Instrument,BL_tau,TROPO_tau,UT_tau,0,1,2,3,4,5,...,146,147,148,149,150,151,152,153,154,155
Time_UTC,AWAS,NaN,NaN,NaN,2014-01-22 01:44:00,2014-01-22 01:55:15,2014-01-22 02:06:30,2014-01-22 02:17:45,2014-01-22 02:29:00,2014-01-22 02:37:56,...,2014-02-22 01:15:43,2014-02-22 01:24:44,2014-02-22 01:33:43,2014-02-22 01:42:44,2014-02-22 01:51:44,2014-02-24 23:36:58,2014-02-24 23:51:51,2014-02-25 00:06:52,2014-02-25 00:21:51,2014-02-25 05:19:51
Flight,AWAS,NaN,NaN,NaN,RF05,RF05,RF05,RF05,RF05,RF05,...,RF14,RF14,RF14,RF14,RF14,RF15,RF15,RF15,RF15,RF15
C2H6_Ethane,AWAS,18.996560,51.867565,170.000000,0.550295,0.555561,0.473938,0.452874,0.731971,0.521332,...,0.651665,0.634551,0.633234,0.605588,0.601638,0.704325,0.738554,0.647716,0.649032,0.662197
C2H2_Ethyne,AWAS,5.460657,9.761061,18.790019,0.360452,0.398395,0.32251,0.32251,0.711419,0.369938,...,0.445823,0.474279,0.483765,0.445823,0.445823,0.55965,0.569135,0.55965,0.597592,0.455308
C3H8_Propane,AWAS,4.276704,9.379963,22.541651,0.195138,0.221748,0.133049,0.133049,0.780552,0.248357,...,0.274967,0.266097,0.257227,0.204008,0.230618,0.310447,0.363666,0.319317,0.301577,0.301577
C4H10_Isobutane,AWAS,2.193483,4.101641,7.916587,0.105958,0.134856,0.0866928,0.0866928,0.992151,0.202283,...,0.154121,0.125223,0.154121,0.0866928,0.154121,0.192651,0.163753,0.11559,0.192651,0.173386
C4H10_nButane,AWAS,1.979710,3.901968,8.096203,0.0946483,0.138817,0.157747,0.119888,1.02851,0.233466,...,0.132508,0.100958,0.132508,0.132508,0.100958,0.157747,0.239776,0.214536,0.201916,0.201916
C5H12_nPentane,AWAS,1.225894,2.360346,4.742530,NaN,NaN,0.0869565,NaN,0.913044,0.0434783,...,0.0869565,NaN,NaN,NaN,NaN,NaN,0.0434783,0.347826,0.0869565,NaN
C6H6_Benzene,AWAS,3.819659,6.602596,12.137543,0.201262,0.214388,0.157509,0.153134,0.686916,0.227514,...,0.310644,0.284392,0.336895,0.301893,0.306268,0.371897,0.411275,0.398149,0.376273,0.319394
OCS_Carbonyl_sulfide,AWAS,2000.000000,2000.000000,2000.000000,1.00577,0.991116,1.00577,1.02592,1.02776,1.01676,...,0.998445,1.00211,0.998445,0.969132,0.93982,0.98562,0.972796,0.956308,0.959972,1.03325


In [12]:
# pickle this!! 
ratios_comb.to_pickle("./contrast_ratios_ut_seg_noRF06.pkl")

In [13]:
##############################################################################################
##############################################################################################

In [14]:
# save tracer names 
# sort 
ratios_sort = ratios_comb
ratios_sort = ratios_sort.drop('Time_UTC').drop('Flight')
ratios_sort = ratios_sort.sort_values(['TROPO_tau'])

# save ordered bl tau as a variable 
tau = ratios_sort['TROPO_tau']

# drop uncecessary columns 
ratios_sort = ratios_sort.drop(columns = ['Instrument', 'BL_tau', 'TROPO_tau', 'UT_tau'])
ratios_sort

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
Acetaldehyde,0.0131152,0.0131152,0.0131152,0.0131152,NaN,0.0131152,NaN,NaN,0.0131152,0.0131152,...,0.082888,0.0131152,0.0131152,0.0131152,0.130365,0.148989,NaN,NaN,NaN,0.0131152
C2H6S_Dimethyl_sulfide,0.00586855,0.00586855,NaN,NaN,NaN,NaN,0,0,0,0.00586855,...,0.0176056,0.00586855,0,0.0117371,NaN,NaN,NaN,NaN,0,NaN
C5H12_nPentane,NaN,NaN,0.0869565,NaN,0.913044,0.0434783,NaN,1.47826,0.0434783,0.217391,...,0.0869565,NaN,NaN,NaN,NaN,NaN,0.0434783,0.347826,0.0869565,NaN
C4H9NO3_n_Butyl_nitrate,0,0,0,0,0.567094,0.122936,0.0991422,0.23001,0,0.341049,...,0,0,0,0,0,0.0436226,0.0713824,0.0317255,0.0118971,0.0793138
n_Butane,0.0667284,0.0904603,0.0209399,0.113355,1.1006,0.331967,0.336155,0.367146,0.280874,0.363516,...,0.132061,0.174499,0.11838,0.0672868,0.134853,0.0921355,NaN,NaN,NaN,0.215262
C4H10_nButane,0.0946483,0.138817,0.157747,0.119888,1.02851,0.233466,0.239776,0.53634,0.233466,0.309184,...,0.132508,0.100958,0.132508,0.132508,0.100958,0.157747,0.239776,0.214536,0.201916,0.201916
i_Butane,0.163329,0.0981525,0.0290966,0.166045,1.00907,0.439165,0.394162,0.327046,0.314243,0.368945,...,0.172252,0.206004,0.258378,0.122594,0.119102,0.436449,NaN,NaN,NaN,0.268077
C4H10_Isobutane,0.105958,0.134856,0.0866928,0.0866928,0.992151,0.202283,0.0288976,0.327506,0.183018,0.346771,...,0.154121,0.125223,0.154121,0.0866928,0.154121,0.192651,0.163753,0.11559,0.192651,0.173386
C4H9NO3_2_Butyl_nitrate,0.522436,0.608209,0.421068,0.506841,0.771958,0.499044,0.366485,0.842136,0.444461,1.10725,...,0.116963,0.0857731,0.109166,0.101368,0.148154,0.124761,0.148154,0.116963,0.070178,0.577019
C6H6_Benzene,0.201262,0.214388,0.157509,0.153134,0.686916,0.227514,0.25814,0.315019,0.271266,0.319394,...,0.310644,0.284392,0.336895,0.301893,0.306268,0.371897,0.411275,0.398149,0.376273,0.319394


In [15]:
# make a thing to fill with tracer names 
trcnames_allseg = pd.DataFrame()
for col in ratios_sort.columns: 
    trcnames_allseg[col] = ''

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155


In [24]:
# loop through each segment 
col = list(ratios_sort)
for i in col:
    utbl_full = np.array(ratios_sort[i].values, dtype=np.float64)
    utbl_not_null_idx = np.argwhere(~np.isnan(utbl_full))
    # ----- inputs without nans 
    my_utbl = utbl_full[utbl_not_null_idx]
    tracer_names = ratios_sort.index[utbl_not_null_idx]
    # ----- fill to make mu*, tau all length 52
    diffa = len(utbl_full) - len(my_utbl)
    if (diffa > 0):
        filla = np.empty((1,diffa))
        filla.fill(np.nan)
        my_utbl = np.append(my_utbl, filla)
        tracer_names = np.append(tracer_names, filla)
    else: 
        #tracer_names = tracer_names.values 
        tracer_names = tracer_names
        #tracer_names = np.ones([52])
        #tracer_names[:] = np.NaN
    # ----- save outputs of tracer names 
    trcnames_allseg[i] = tracer_names
#trcnames_allseg

/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Support for multi-dimensional indexing (e.g. `index[:, None]`) on an Index is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  


In [17]:
trcnames_allseg.to_pickle("./trcnames_allseg_trtau_noRF06.pkl")